In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load gdsc2


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数

def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))
    
    return fold_results

# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]
    
    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task) for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )
    
    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)], 
                ignore_index=True,
                copy=False  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)], 
                ignore_index=True,
                copy=False
            )


  0%|          | 0/910 [00:00<?, ?it/s]

epoch:   0 loss:0.697897 auc:0.6123
epoch:  20 loss:0.152424 auc:0.9678
epoch:  40 loss:0.135410 auc:0.9785
epoch:  60 loss:0.121891 auc:0.9814
epoch:  80 loss:0.113669 auc:0.9893
epoch: 100 loss:0.107522 auc:0.9912
epoch: 120 loss:0.101367 auc:0.9932
epoch: 140 loss:0.096217 auc:0.9922
epoch: 160 loss:0.091487 auc:0.9951
epoch: 180 loss:0.090031 auc:0.9941
epoch: 200 loss:0.085571 auc:0.9951
epoch: 220 loss:0.084367 auc:0.9951
epoch: 240 loss:0.081283 auc:0.9951
epoch: 260 loss:0.079638 auc:0.9961
epoch: 280 loss:0.079228 auc:0.9951
epoch: 300 loss:0.077787 auc:0.9951
epoch: 320 loss:0.076982 auc:0.9941
epoch: 340 loss:0.076436 auc:0.9961
epoch: 360 loss:0.075708 auc:0.9971
epoch: 380 loss:0.076548 auc:0.9971
epoch: 400 loss:0.075110 auc:0.9971
epoch: 420 loss:0.075228 auc:0.9971
epoch: 440 loss:0.074357 auc:0.9971
epoch: 460 loss:0.075342 auc:0.9961
epoch: 480 loss:0.074008 auc:0.9951
epoch: 500 loss:0.075495 auc:0.9951
epoch: 520 loss:0.075346 auc:0.9951
epoch: 540 loss:0.073823 auc

  0%|          | 1/910 [00:22<5:36:13, 22.19s/it]

Fit finished.
epoch:   0 loss:0.698901 auc:0.5527
epoch:  20 loss:0.153658 auc:0.9682
epoch:  40 loss:0.137886 auc:0.9796
epoch:  60 loss:0.124283 auc:0.9780
epoch:  80 loss:0.115294 auc:0.9780
epoch: 100 loss:0.109108 auc:0.9812
epoch: 120 loss:0.103093 auc:0.9788
epoch: 140 loss:0.097517 auc:0.9731
epoch: 160 loss:0.093098 auc:0.9714
epoch: 180 loss:0.088810 auc:0.9706
epoch: 200 loss:0.086265 auc:0.9600
epoch: 220 loss:0.083468 auc:0.9641
epoch: 240 loss:0.081437 auc:0.9592
epoch: 260 loss:0.080430 auc:0.9633
epoch: 280 loss:0.078943 auc:0.9657
epoch: 300 loss:0.078334 auc:0.9600
epoch: 320 loss:0.077191 auc:0.9633
epoch: 340 loss:0.076918 auc:0.9657
epoch: 360 loss:0.076170 auc:0.9641
epoch: 380 loss:0.075774 auc:0.9633
epoch: 400 loss:0.075125 auc:0.9624
epoch: 420 loss:0.074744 auc:0.9616
epoch: 440 loss:0.074952 auc:0.9641
epoch: 460 loss:0.074433 auc:0.9649
epoch: 480 loss:0.073942 auc:0.9641
epoch: 500 loss:0.074245 auc:0.9641
epoch: 520 loss:0.075725 auc:0.9624
epoch: 540 los

  0%|          | 2/910 [00:40<5:03:58, 20.09s/it]

...

  6%|▋         | 57/910 [36:21<14:25:21, 60.87s/it]

Fit finished.
epoch:   0 loss:0.692800 auc:0.3401
epoch:  20 loss:0.153735 auc:1.0000
epoch:  40 loss:0.137536 auc:1.0000
epoch:  60 loss:0.123996 auc:1.0000
epoch:  80 loss:0.115236 auc:1.0000
epoch: 100 loss:0.109173 auc:1.0000
epoch: 120 loss:0.103316 auc:1.0000
epoch: 140 loss:0.097782 auc:1.0000
epoch: 160 loss:0.093010 auc:1.0000
epoch: 180 loss:0.089294 auc:1.0000
epoch: 200 loss:0.086318 auc:1.0000
epoch: 220 loss:0.083380 auc:1.0000
epoch: 240 loss:0.083939 auc:1.0000
epoch: 260 loss:0.080248 auc:1.0000
epoch: 280 loss:0.079065 auc:1.0000
epoch: 300 loss:0.077622 auc:1.0000
epoch: 320 loss:0.079053 auc:1.0000
epoch: 340 loss:0.076545 auc:1.0000
epoch: 360 loss:0.075743 auc:1.0000
epoch: 380 loss:0.076006 auc:1.0000
epoch: 400 loss:0.074957 auc:1.0000
epoch: 420 loss:0.075188 auc:1.0000
epoch: 440 loss:0.075270 auc:1.0000
epoch: 460 loss:0.074198 auc:1.0000
epoch: 480 loss:0.074446 auc:1.0000
epoch: 500 loss:0.074171 auc:1.0000
epoch: 520 loss:0.073582 auc:1.0000
epoch: 540 los

  6%|▋         | 58/910 [37:21<14:20:06, 60.57s/it]

Fit finished.
epoch:   0 loss:0.703704 auc:0.4809
epoch:  20 loss:0.153245 auc:0.9640
epoch:  40 loss:0.136552 auc:0.9701
epoch:  60 loss:0.122851 auc:0.9709
epoch:  80 loss:0.114203 auc:0.9683
epoch: 100 loss:0.108078 auc:0.9692
epoch: 120 loss:0.102086 auc:0.9688
epoch: 140 loss:0.096505 auc:0.9692
epoch: 160 loss:0.091915 auc:0.9714
epoch: 180 loss:0.088079 auc:0.9718
epoch: 200 loss:0.085406 auc:0.9701


  6%|▋         | 58/910 [37:35<9:12:07, 38.88s/it] 


In [ ]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")